## Join the Duet Server the Data Owner 1 connected to

In [2]:
import pandas as pd

In [1]:
import syft as sy
duet1 = sy.duet("530dcaacef2fc6a6680a4f58d28fb0e3")

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following Duet Client ID to your duet partner!
♫♫♫ > Duet Client ID: 5050b6dbf257aa4107f24dc658a7af92

♫♫♫ > ...waiting for partner to connect...


C:\Users\dylan\anaconda3\envs\new_pysyft\lib\site-packages\aiortc\rtcdtlstransport.py:211: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  _openssl_assert(lib.SSL_CTX_use_certificate(ctx, self._cert._x509) == 1)  # type: ignore
C:\Users\dylan\anaconda3\envs\new_pysyft\lib\site-packages\aiortc\rtcdtlstransport.py:186: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  value=certificate_digest(self._cert._x509),  # type: ignore



♫♫♫ > CONNECTED!


## Join the Duet Server the Data Owner 2 connected to

In [3]:
duet2 = sy.duet("c428f674d926eb340a98d6ce00d07bfd")

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following Duet Client ID to your duet partner!
♫♫♫ > Duet Client ID: 8c8a328b3505430e2e119ad76cada7ea

♫♫♫ > ...waiting for partner to connect...

♫♫♫ > CONNECTED!


In [4]:
duet1.store.pandas

,ID,Tags,Description,object_type
0,<UID: ce63eddb907943f4a2040b960a2140ec>,[DO1 training X data],number of samples and features,<class 'torch.Tensor'>
1,<UID: 04cb77c0b4734e5881c09a1c48f8697e>,[DO1 training y data],number of samples and features,<class 'torch.Tensor'>


In [5]:
duet2.store.pandas

,ID,Tags,Description,object_type
0,<UID: 9d473476b9334f4bad413cca4a823e54>,[DO2 training X data],number of samples and features,<class 'torch.Tensor'>
1,<UID: 098db24261474e639bf592added98ba1>,[DO1 training y data],number of samples and features,<class 'torch.Tensor'>
2,<UID: cdc9f879951b461b94b9329afc830296>,[DO2 training y data],number of samples and features,<class 'torch.Tensor'>


## Modelling

In [6]:
data1_ptr = duet1.store[0]
target1_ptr = duet1.store[1]
data2_ptr = duet2.store[0]
target2_ptr = duet2.store[1]

print(data1_ptr)
print(target1_ptr)
print(data2_ptr)
print(target2_ptr)

### Create Base Model

In [7]:
import torch
import torch.nn.functional as F
import torch.nn as nn 
from torch import optim
from torch.utils.data import Dataset, DataLoader

In [5]:
#in_dim = 39
#out_dim = 1

In [8]:
#class SyNet(sy.Module):
    #def __init__(self, torch_ref):
        #super(SyNet, self).__init__(torch_ref=torch_ref)
        #self.linear = self.torch_ref.nn.Linear(in_dim, out_dim)

    #def forward(self, x):
        #x = self.linear(x)
        #return x

In [9]:
class SyNet(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet, self).__init__(torch_ref=torch_ref)
        
        self.layer_1 = self.torch_ref.nn.Linear(39, 31)
        self.layer_out = self.torch_ref.nn.Linear(31, 1)
        # Define proportion or neurons to dropout
        self.dropout_1 = self.torch_ref.nn.Dropout(0.12409392594394411)
        self.relu = self.torch_ref.nn.ReLU()
    
    def forward(self, inputs):
        x = self.dropout_1(self.relu(self.layer_1(inputs)))
        x = self.layer_out(x)

        return (x)

    def predict(self, test_inputs):
        x = self.relu(self.layer_1(test_inputs))
        x = self.layer_out(x)

        return (x)

### Training

In [10]:
def train(iterations, model, torch_ref, optim, data_ptr, target_ptr):

    #criterion = torch_ref.nn.functional.poisson_nll_loss(log_input= True, full= True)

    losses = []

    for i in range(iterations):

        optim.zero_grad()

        output = model(data_ptr)

        #loss = torch_ref.nn.functional.nll_loss(output, target_ptr)
        loss = torch_ref.nn.functional.poisson_nll_loss(output, target_ptr, log_input= True, full= True)   
        #loss = torch_ref.nn.functional.mse_loss(output, target_ptr)
        #loss = criterion(output, target_ptr)

        loss_item = loss.item()

        loss_value = loss_item.get(
            reason="To evaluate training progress",
            request_block=True,
            timeout_secs=5,
        )

        print("Epoch", i, "loss", loss_value)

        losses.append(loss_value)

        loss.backward()

        optim.step()

    return losses

#### Send one copy of the model to each data owner or client and train them remotely one by one

In [11]:
import torch as th
import numpy as np

In [12]:
base_model = SyNet(torch)

Train on Data Owner 1 data

In [13]:
remote_model1 = base_model.send(duet1)

In [14]:
remote_torch1 = duet1.torch
params = remote_model1.parameters()
optim1 = remote_torch1.optim.Adam(params=params, lr=0.1)

In [15]:
iteration = 25
losses = train(iteration, remote_model1, remote_torch1, optim1, data1_ptr, target1_ptr)

Epoch 0 loss 0.9062418341636658
Epoch 1 loss 0.24310307204723358
Epoch 2 loss 0.2929839491844177
Epoch 3 loss 0.3667319715023041
Epoch 4 loss 0.36441570520401
Epoch 5 loss 0.31841129064559937
Epoch 6 loss 0.26185107231140137
Epoch 7 loss 0.22236983478069305
Epoch 8 loss 0.22423219680786133
Epoch 9 loss 0.25723570585250854
Epoch 10 loss 0.2569369077682495
Epoch 11 loss 0.23286843299865723
Epoch 12 loss 0.21849031746387482
Epoch 13 loss 0.21901684999465942
Epoch 14 loss 0.22683142125606537
Epoch 15 loss 0.23350588977336884
Epoch 16 loss 0.23508597910404205
Epoch 17 loss 0.23126232624053955
Epoch 18 loss 0.22396337985992432
Epoch 19 loss 0.21785898506641388
Epoch 20 loss 0.21519169211387634
Epoch 21 loss 0.21713753044605255
Epoch 22 loss 0.22117801010608673
Epoch 23 loss 0.2233625054359436
Epoch 24 loss 0.22200001776218414


Train on Data Owner 2 data

In [16]:
remote_model2 = base_model.send(duet2)

In [17]:
remote_torch2 = duet2.torch
params = remote_model2.parameters()
optim2 = remote_torch2.optim.Adam(params=params, lr=0.1)

In [18]:
iteration = 25
losses = train(iteration, remote_model2, remote_torch2, optim2, data2_ptr, target2_ptr)

Epoch 0 loss 0.9064223766326904
Epoch 1 loss 0.24471142888069153
Epoch 2 loss 0.2971583604812622
Epoch 3 loss 0.3697357177734375
Epoch 4 loss 0.3653009533882141
Epoch 5 loss 0.31817325949668884
Epoch 6 loss 0.26193881034851074
Epoch 7 loss 0.22358867526054382
Epoch 8 loss 0.22770382463932037
Epoch 9 loss 0.260432630777359
Epoch 10 loss 0.2579480707645416
Epoch 11 loss 0.2338380664587021
Epoch 12 loss 0.22008924186229706
Epoch 13 loss 0.221822589635849
Epoch 14 loss 0.2299414724111557
Epoch 15 loss 0.23647285997867584
Epoch 16 loss 0.2374122142791748
Epoch 17 loss 0.23362837731838226
Epoch 18 loss 0.2257436364889145
Epoch 19 loss 0.21958300471305847
Epoch 20 loss 0.2174249142408371
Epoch 21 loss 0.21976684033870697
Epoch 22 loss 0.22367244958877563
Epoch 23 loss 0.22623446583747864
Epoch 24 loss 0.22400985658168793


### Averaging Model Updates

Ideally, there will be a coordinator server with a secure aggreagtor who will get the model updates from different clients and make an aggregation. For the case of simplicity, in this example we will make the Data Sceintist server work as the coordinator.

### Little sanity check!

In [19]:
param1 = remote_model1.parameters().get(request_block=True)
param2 = remote_model2.parameters().get(request_block=True)

print("Base Model parameters:")
print(base_model.parameters())
print()

print("Remote model1 parameters:")
print(param1)
print()

print("Remote model2 parameters:")
print(param2)

Base Model parameters:
[Parameter containing:
tensor([[-0.1070, -0.0385, -0.1026,  ..., -0.0935, -0.1478,  0.0914],
        [ 0.0548,  0.1183, -0.0727,  ..., -0.0354, -0.0481, -0.0328],
        [ 0.0441, -0.0070, -0.0527,  ..., -0.0754, -0.0846, -0.0453],
        ...,
        [-0.1439, -0.1211, -0.0714,  ..., -0.1552, -0.0535,  0.1165],
        [ 0.0562,  0.1211, -0.0228,  ..., -0.0194,  0.0147,  0.1411],
        [-0.0275,  0.1053,  0.0532,  ..., -0.0454,  0.0620, -0.0108]],
       requires_grad=True), Parameter containing:
tensor([-0.1388,  0.0973,  0.0552,  0.1592, -0.0140, -0.0576,  0.1161, -0.0709,
         0.0212,  0.1106,  0.1194, -0.1148,  0.1299, -0.0138,  0.0219,  0.0495,
        -0.1123, -0.0955,  0.1345, -0.0425, -0.1319, -0.0545,  0.1184,  0.0525,
         0.1382,  0.1225,  0.0090,  0.0927,  0.0958, -0.0975,  0.1142],
       requires_grad=True), Parameter containing:
tensor([[-0.0989, -0.0936,  0.1435,  0.1719,  0.0985, -0.0420,  0.0496, -0.0984,
          0.0653, -0.1332, 

As you can see, the remote model paramter values are different from the base model paramter values. That means the remote copies of our base model got trained and updated.

In [20]:
remote_model1_updates = remote_model1.get(
    request_block=True
).state_dict()

print(remote_model1_updates)

OrderedDict([('layer_1.weight', tensor([[-0.1670,  0.1333,  0.0982,  ...,  0.3623,  0.0920,  0.4207],
        [ 0.0253,  0.2840,  0.0993,  ...,  0.4003,  0.2503,  0.2506],
        [-0.5125, -0.5637, -0.6093,  ..., -0.6320, -0.6412, -0.6014],
        ...,
        [-0.4123, -0.1489, -0.0521,  ..., -0.0902,  0.0187,  0.3375],
        [-0.5005, -0.4355, -0.5794,  ..., -0.5760, -0.5419, -0.4155],
        [-0.4571, -0.1743, -0.2296,  ..., -0.2052, -0.0750, -0.3743]])), ('layer_1.bias', tensor([ 0.1075,  0.3245, -0.5014, -0.3974, -0.5707, -0.2693, -0.4405,  0.1827,
        -0.5354, -0.1769, -0.4372, -0.3892,  0.4855, -0.3706, -0.5347,  0.3754,
        -0.6683, -0.6522, -0.4210, -0.5991, -0.6885, -0.6111,  0.3191, -0.5041,
         0.1814, -0.4341, -0.3128, -0.4639,  0.1495, -0.6542, -0.1471])), ('layer_out.weight', tensor([[-0.1914, -0.1612, -0.4131, -0.3847, -0.4581,  0.1123, -0.5070, -0.1808,
         -0.4913,  0.4729, -0.5348,  0.5467, -0.2107,  0.3628, -0.4457, -0.1999,
         -0.5385, 

In [21]:
remote_model2_updates = remote_model2.get(
    request_block=True
).state_dict()

print(remote_model2_updates)

OrderedDict([('layer_1.weight', tensor([[-0.1647,  0.1446,  0.1084,  ...,  0.3582,  0.0575,  0.3988],
        [ 0.0259,  0.2932,  0.1088,  ...,  0.3948,  0.2146,  0.2566],
        [-0.5125, -0.5637, -0.6093,  ..., -0.6320, -0.6412, -0.6013],
        ...,
        [-0.4098, -0.1406, -0.0457,  ..., -0.0980, -0.0257,  0.3207],
        [-0.5005, -0.4355, -0.5794,  ..., -0.5760, -0.5419, -0.4155],
        [-0.4580, -0.1692, -0.2266,  ..., -0.2149, -0.1085, -0.3907]])), ('layer_1.bias', tensor([ 0.1087,  0.3240, -0.5014, -0.3974, -0.5707, -0.2702, -0.4405,  0.1822,
        -0.5354, -0.1807, -0.4372, -0.3927,  0.4832, -0.3691, -0.5347,  0.3747,
        -0.6683, -0.6522, -0.4210, -0.5991, -0.6885, -0.6111,  0.3178, -0.5041,
         0.1824, -0.4341, -0.3156, -0.4639,  0.1468, -0.6542, -0.1500])), ('layer_out.weight', tensor([[-0.1952, -0.1631, -0.4131, -0.3847, -0.4581,  0.1143, -0.5070, -0.1814,
         -0.4913,  0.4712, -0.5348,  0.5464, -0.2102,  0.3655, -0.4457, -0.1999,
         -0.5385, 

In [22]:
from collections import OrderedDict

Let's do the aggregation of the weights. In this example, we will just calculate the average of corresponding weights from each model.

In [25]:
avg_updates = OrderedDict()
avg_updates["layer_1.weight"] = (
    remote_model1_updates["layer_1.weight"] + remote_model2_updates["layer_1.weight"]
) / 2
avg_updates["layer_1.bias"] = (
    remote_model1_updates["layer_1.bias"] + remote_model2_updates["layer_1.bias"]
) / 2
avg_updates["layer_out.weight"] = (
    remote_model1_updates["layer_out.weight"] + remote_model2_updates["layer_out.weight"]
) / 2
avg_updates["layer_out.bias"] = (
    remote_model1_updates["layer_out.bias"] + remote_model2_updates["layer_out.bias"]
) / 2

print(avg_updates)

OrderedDict([('layer_1.weight', tensor([[-0.1658,  0.1390,  0.1033,  ...,  0.3603,  0.0748,  0.4097],
        [ 0.0256,  0.2886,  0.1040,  ...,  0.3975,  0.2324,  0.2536],
        [-0.5125, -0.5637, -0.6093,  ..., -0.6320, -0.6412, -0.6013],
        ...,
        [-0.4110, -0.1448, -0.0489,  ..., -0.0941, -0.0035,  0.3291],
        [-0.5005, -0.4355, -0.5794,  ..., -0.5760, -0.5419, -0.4155],
        [-0.4576, -0.1717, -0.2281,  ..., -0.2101, -0.0918, -0.3825]])), ('layer_1.bias', tensor([ 0.1081,  0.3242, -0.5014, -0.3974, -0.5707, -0.2697, -0.4405,  0.1824,
        -0.5354, -0.1788, -0.4372, -0.3910,  0.4844, -0.3698, -0.5347,  0.3751,
        -0.6683, -0.6522, -0.4210, -0.5991, -0.6885, -0.6111,  0.3184, -0.5041,
         0.1819, -0.4341, -0.3142, -0.4639,  0.1482, -0.6542, -0.1486])), ('layer_out.weight', tensor([[-0.1933, -0.1622, -0.4131, -0.3847, -0.4581,  0.1133, -0.5070, -0.1811,
         -0.4913,  0.4721, -0.5348,  0.5466, -0.2105,  0.3641, -0.4457, -0.1999,
         -0.5385, 

### Load aggregated weights

In [26]:
combined_model = SyNet(torch)

In [27]:
combined_model.load_state_dict(avg_updates)

In [28]:
del avg_updates, remote_model1_updates, remote_model2_updates

In [29]:
df_X_val_0 = pd.read_csv('ETL/X_val_0.csv')
df_y_val_0 = pd.read_csv('ETL/y_val_0.csv')
X_val_0_np = df_X_val_0.to_numpy()
y_val_0_np = df_y_val_0.to_numpy()
X_val_0 = th.from_numpy(X_val_0_np).float()
y_val_0 = th.from_numpy(y_val_0_np).float()

In [30]:
preds = []
with torch.no_grad():
    for i in range(len(X_val_0)):
        sample = X_val_0[i]
        y_hat = combined_model(sample)

        print(f"Prediction: {y_hat.item()} Ground Truth: {y_val_0[i].item()}")
        preds.append(y_hat)

Prediction: -2.0626211166381836 Ground Truth: 0.0
Prediction: -2.4296646118164062 Ground Truth: 0.0
Prediction: -2.383727550506592 Ground Truth: 0.0
Prediction: -2.3148794174194336 Ground Truth: 0.0
Prediction: -2.627004623413086 Ground Truth: 0.0
Prediction: -2.3470144271850586 Ground Truth: 0.0
Prediction: -2.9395227432250977 Ground Truth: 0.0
Prediction: -3.010042667388916 Ground Truth: 1.0
Prediction: -3.121812343597412 Ground Truth: 0.0
Prediction: -3.2139854431152344 Ground Truth: 0.0
Prediction: -2.595306396484375 Ground Truth: 0.0
Prediction: -2.364854335784912 Ground Truth: 0.0
Prediction: -2.872859477996826 Ground Truth: 0.0
Prediction: -2.6387743949890137 Ground Truth: 2.0
Prediction: -2.417076587677002 Ground Truth: 0.0
Prediction: -1.7589703798294067 Ground Truth: 0.0
Prediction: -2.873605251312256 Ground Truth: 0.0
Prediction: -2.7078287601470947 Ground Truth: 0.0
Prediction: -2.7457809448242188 Ground Truth: 0.0
Prediction: -2.1966772079467773 Ground Truth: 0.0
Predictio

In [31]:
preds = []
with torch.no_grad():
    for i in range(len(X_val_0)):
        sample = X_val_0[i]
        y_hat = base_model(sample)

        print(f"Prediction: {y_hat.item()} Ground Truth: {y_val_0[i].item()}")
        preds.append(y_hat)

Prediction: -0.03796558082103729 Ground Truth: 0.0
Prediction: -0.14589761197566986 Ground Truth: 0.0
Prediction: -0.07810802012681961 Ground Truth: 0.0
Prediction: -0.037573061883449554 Ground Truth: 0.0
Prediction: -0.15101082623004913 Ground Truth: 0.0
Prediction: -0.08674273639917374 Ground Truth: 0.0
Prediction: -0.1281956434249878 Ground Truth: 0.0
Prediction: -0.025626391172409058 Ground Truth: 1.0
Prediction: -0.14142757654190063 Ground Truth: 0.0
Prediction: -0.1180603951215744 Ground Truth: 0.0
Prediction: -0.07446727156639099 Ground Truth: 0.0
Prediction: -0.10321163386106491 Ground Truth: 0.0
Prediction: -0.11325041204690933 Ground Truth: 0.0
Prediction: -0.17834044992923737 Ground Truth: 2.0
Prediction: -0.11292323470115662 Ground Truth: 0.0
Prediction: -0.13577142357826233 Ground Truth: 0.0
Prediction: -0.21868456900119781 Ground Truth: 0.0
Prediction: -0.09458325803279877 Ground Truth: 0.0
Prediction: -0.1813318431377411 Ground Truth: 0.0
Prediction: -0.18731194734573364

## Comparison to classical linear regression on centralised data

In [ ]:
import torch
import numpy as np

in_dim = 1
out_dim = 1


class ClassicalLR(torch.nn.Module):
    def __init__(self, torch):
        super(ClassicalLR, self).__init__()
        self.linear = torch.nn.Linear(in_dim, out_dim)

    def forward(self, x):
        x = self.linear(x)
        return x


classical_model = ClassicalLR(torch)

In [ ]:
data = torch.FloatTensor(
    np.array([5, 15, 25, 35, 45, 55, 60, 65, 75, 85, 95]).reshape(-1, 1)
)
target = torch.FloatTensor(
    np.array([5, 10, 15, 22, 30, 38, 35, 40, 45, 55, 60]).reshape(-1, 1)
)

In [ ]:
def classic_train(iterations, model, torch, optim, data, target, criterion):

    losses = []

    for i in range(iterations):

        optim.zero_grad()

        output = model(data)

        loss = criterion(output, target)

        loss_item = loss.item()

        if i % 10 == 0:
            print("Epoch", i, "loss", loss_item)

        losses.append(loss_item)

        loss.backward()

        optim.step()

    return losses

In [ ]:
params = classical_model.parameters()
optim = torch.optim.Adam(params=params, lr=0.1)
criterion = torch.nn.MSELoss()

In [ ]:
iteration = 100
losses = classic_train(
    iteration, classical_model, torch, optim, data, target, criterion
)

In [ ]:
test_data = th.FloatTensor(np.array([17, 25, 32, 50, 80]).reshape(-1, 1))
test_target = th.FloatTensor(np.array([12, 15, 20, 30, 50]).reshape(-1, 1))

In [ ]:
preds = []
with torch.no_grad():
    for i in range(len(test_data)):
        sample = test_data[i]
        y_hat = classical_model(sample)

        print(f"Prediction: {y_hat.item()} Ground Truth: {test_target[i].item()}")
        preds.append(y_hat)